# Gliderpy

gliderpy builds off of the functionality of erddapy to make glider data access, manipulation, and visualization even simpler 

(https://github.com/oceanhackweek/ohw20-proj-glide-data-fetcher) 

# Here is an example of plain erddapy

Copied from the [errdapy intro](https://ioos.github.io/erddapy/quick_intro-output.html#Longer-introduction)

Erddapy is great but we want to try to make it even more simple with fewer lines of code with our **gliderpy** package

In [ ]:
from erddapy import ERDDAP


e = ERDDAP(
    server="https://gliders.ioos.us/erddap",
    protocol="tabledap",
    response="csv",
)
e.dataset_id = "whoi_406-20160902T1700"

e.variables = [
    "depth",
    "latitude",
    "longitude",
    "salinity",
    "temperature",
    "time",
]

e.constraints = {
    "time>=": "2016-07-10T00:00:00Z",
    "time<=": "2017-02-10T00:00:00Z",
    "latitude>=": 38.0,
    "latitude<=": 41.0,
    "longitude>=": -72.0,
    "longitude<=": -69.0,
}


In [ ]:
df = e.to_pandas(
    index_col="time (UTC)",
    parse_dates=True,
)
df.head()

# Fetch glider data using glider ID
Emulating the UI of [argopy](https://github.com/euroargodev/argopy)

In [ ]:
from gliderpy.fetchers import GliderDataFetcher

glider_grab = GliderDataFetcher()

glider_grab.fetcher.dataset_id = "whoi_406-20160902T1700"
df = glider_grab.to_pandas()
df.head()

Try with a different dataset

In [ ]:
glider_grab = GliderDataFetcher()
glider_grab.fetcher.dataset_id = "sp022-20170209T1616"

df = glider_grab.to_pandas()
df.head()

# Now we will pass time and space constraints
We can make a direct call to erddapy's `constraints'

In [ ]:
glider_grab = GliderDataFetcher()

glider_grab.fetcher.dataset_id = "whoi_406-20160902T1700"

glider_grab.fetcher.constraints = {
    "time>=": "2016-07-10T00:00:00Z",
    "time<=": "2016-09-15T00:00:00Z",
    "latitude>=": 38.0,
    "latitude<=": 41.0,
    "longitude>=": -72.0,
    "longitude<=": -69.0,
}

df = glider_grab.to_pandas()
df.tail()

Let's try making a method of the class to do this more neatly

In [ ]:
glider_grab = GliderDataFetcher()

# In this case we don't want to specify a dataset_id, we want all gliders in that query.
glider_grab.fetcher.dataset_id = "whoi_406-20160902T1700"
min_lat, max_lat, min_lon, max_lon = 10, 40.5, -71, 180
start_time, end_time = "2016-08-1","2016-09-18"
glider_grab.query(
    min_lat,
    max_lat,
    min_lon,
    max_lon,
    start_time,
    end_time
)

df = glider_grab.to_pandas()
df.tail()

In [ ]:
df.describe()

How to ind glider datasets on server

Calling get_ids from the class searches for all databases tagged as glider

In [ ]:
from gliderpy.fetchers import DatasetList

datasets = DatasetList()
datasets.get_ids()

Let's look for the ones from UW

In [ ]:
datasets = DatasetList()
datasets.search_terms = ["UW"]
uw_glider_ids = datasets.get_ids()
uw_glider_ids

We'll pass that id back into our original data fetcher

In [ ]:
uw_grab = GliderDataFetcher()
uw_grab.fetcher.dataset_id = uw_glider_ids[18]

df = uw_grab.to_pandas(
)
df.head()

## Let's plot the track map for a dataset

In [ ]:
#Colored by temperature--will add kwargs in a future update
plot_track(df)

## Now the transect for a user defined variable in that dataset

In [ ]:
#Variable options: salinity (1) or temperature (Celcius)
plot_transect(df,"temperature (Celcius)")

In [ ]:
plot_transect(df,"salinity (1)")